<a href="https://colab.research.google.com/github/sushmitha-pothuganti/22BK1A6694/blob/main/facial_expression_recognition_for_restaurent_feed_back_(working).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gradio deepface opencv-python pandas plotly pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
# restaurant_sentiment_app.py
import os
import sys
import json
from datetime import datetime
import tempfile

# Install helper if packages missing (optional)
def ensure_pkg(pkg):
    try:
        __import__(pkg)
        return True
    except ImportError:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        return True

for p in ["gradio", "deepface", "opencv-python", "pandas", "plotly", "Pillow"]:
    ensure_pkg(p)

import gradio as gr
from deepface import DeepFace
import numpy as np
from PIL import Image
import cv2
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# ---------- Configuration ----------
REVIEWS_DIR = "customer_reviews"
REVIEWS_JSON = os.path.join(REVIEWS_DIR, "emotion_reviews.json")
REVIEWS_CSV = os.path.join(REVIEWS_DIR, "emotion_reviews.csv")

os.makedirs(REVIEWS_DIR, exist_ok=True)

# Initialize files
if not os.path.exists(REVIEWS_JSON):
    with open(REVIEWS_JSON, "w") as f:
        json.dump([], f, indent=2)

if not os.path.exists(REVIEWS_CSV):
    import csv
    with open(REVIEWS_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "date", "time", "image_name", "dominant_emotion", "confidence", "feedback_type"])

# ---------- Storage helpers ----------
def load_reviews_json():
    try:
        with open(REVIEWS_JSON, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return []

def append_review_to_files(review: dict):
    # JSON
    reviews = load_reviews_json()
    reviews.append(review)
    with open(REVIEWS_JSON, "w", encoding="utf-8") as f:
        json.dump(reviews, f, indent=2, ensure_ascii=False)

    # CSV
    import csv
    with open(REVIEWS_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            review.get("timestamp"),
            review.get("date"),
            review.get("time"),
            review.get("image_name"),
            review.get("dominant_emotion"),
            review.get("confidence"),
            review.get("feedback_type")
        ])

# ---------- Emotion detection & mapping ----------
def image_to_tempfile(img_pil):
    """Save PIL image to temp file path and return path."""
    if img_pil is None:
        return None
    if isinstance(img_pil, np.ndarray):
        img_pil = Image.fromarray(img_pil[..., ::-1])  # BGR->RGB
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".jpg")
    img_pil.save(tmp.name, format="JPEG")
    tmp.close()
    return tmp.name

def interpret_emotion(dominant_emotion):
    """Map DeepFace emotion to restaurant command and feedback type."""
    pos = ['happy', 'surprise']
    neg = ['sad', 'angry', 'disgust', 'fear']
    neutral = ['neutral']

    if dominant_emotion in pos:
        if dominant_emotion == "happy":
            return ("😊 Customer satisfied — Serve next dish.", "Positive")
        else:
            return ("😮 Customer surprised — check if dish exceeded expectations.", "Positive")
    elif dominant_emotion in neg:
        if dominant_emotion == "angry":
            return ("😡 Customer angry — alert staff immediately.", "Negative")
        elif dominant_emotion == "disgust":
            return ("🤢 Possible food quality issue — inspect dish & offer replacement.", "Negative")
        elif dominant_emotion == "sad":
            return ("😔 Customer disappointed — ask for feedback / offer help.", "Negative")
        else: # fear
            return ("😰 Customer uncomfortable — check safety & assist.", "Negative")
    else:
        return ("😐 Neutral — continue normal service.", "Neutral")

def detect_expression(image):
    """
    image: PIL.Image or numpy array (as provided by gr.Image with type="pil").
    Returns: (emotion_text, command_text, recommendation_text, dominant_emotion)
    Also saves review to JSON and CSV automatically.
    """
    try:
        if image is None:
            return ("❌ No image provided", "Please upload or capture a face image", "", "")

        # Ensure PIL -> temp file path for DeepFace
        img_path = image_to_tempfile(image)

        # DeepFace analyze
        result = DeepFace.analyze(img_path=img_path, actions=['emotion'], enforce_detection=False, silent=True)

        # DeepFace may return dict or list (multi-face). Handle both.
        if isinstance(result, list) and len(result) > 0:
            result = result[0]

        dominant_emotion = result.get("dominant_emotion", "neutral")
        emotions_scores = result.get("emotion", {})
        confidence = emotions_scores.get(dominant_emotion, 0)

        # Map to restaurant command
        command_text, feedback_type = interpret_emotion(dominant_emotion)

        # Prepare readable emotion details
        top3 = sorted(emotions_scores.items(), key=lambda x: x[1], reverse=True)[:3]
        emotion_details = f"🎭 {dominant_emotion.title()} ({confidence:.1f}%)\nTop: " + ", ".join([f"{e}: {s:.1f}%" for e, s in top3])

        # Save review
        ts = datetime.now().isoformat()
        review = {
            "timestamp": ts,
            "date": datetime.now().strftime("%Y-%m-%d"),
            "time": datetime.now().strftime("%H:%M:%S"),
            "image_name": os.path.basename(img_path),
            "dominant_emotion": dominant_emotion,
            "confidence": round(float(confidence), 2),
            "feedback_type": feedback_type
        }
        append_review_to_files(review)

        # Clean temp file
        try:
            os.unlink(img_path)
        except Exception:
            pass

        return (emotion_details, f"{command_text}", f"Recommendation: {command_text}", dominant_emotion)

    except Exception as e:
        return (f"❌ Analysis Error: {str(e)}", "Processing failed", "", "")

# ---------- Analytics ----------
def create_analytics():
    """Read CSV/JSON and create summary + plotly figures"""
    try:
        if not os.path.exists(REVIEWS_CSV):
            return ("No data yet. Run some analyses first!", None, None, None, None)

        df = pd.read_csv(REVIEWS_CSV)
        if df.empty:
            return ("No data yet. Run some analyses first!", None, None, None, None)

        # Basic stats
        total = len(df)
        pos = len(df[df['feedback_type'] == 'Positive'])
        neg = len(df[df['feedback_type'] == 'Negative'])
        neu = len(df[df['feedback_type'] == 'Neutral'])
        avg_conf = df['confidence'].mean()

        summary = f"""
**Total Reviews:** {total}
• Positive: {pos}
• Negative: {neg}
• Neutral: {neu}
**Average Confidence:** {avg_conf:.1f}%
Latest: {df['timestamp'].iloc[-1]}
"""

        # Sentiment pie
        pie_df = pd.DataFrame({
            "Sentiment": ["Positive", "Negative", "Neutral"],
            "Count": [pos, neg, neu]
        })
        fig_pie = px.pie(pie_df, names="Sentiment", values="Count", title="Sentiment Distribution")
        fig_pie.update_traces(textinfo='percent+label')

        # Emotion counts bar
        emo_counts = df['dominant_emotion'].value_counts().reset_index()
        emo_counts.columns = ['Emotion', 'Count']
        fig_bar = px.bar(emo_counts, x='Emotion', y='Count', title="Emotion Counts")

        # Daily trend (if date exists)
        try:
            df['date_parsed'] = pd.to_datetime(df['date'])
            daily = df.groupby([df['date_parsed'].dt.date, 'feedback_type']).size().unstack(fill_value=0)
            fig_trend = go.Figure()
            for col in daily.columns:
                fig_trend.add_trace(go.Scatter(x=list(daily.index), y=daily[col], mode='lines+markers', name=col))
            fig_trend.update_layout(title="Daily Sentiment Trend", xaxis_title="Date", yaxis_title="Count")
        except Exception:
            fig_trend = None

        # Confidence histogram
        fig_conf = px.histogram(df, x='confidence', nbins=20, title="Confidence Score Distribution")

        return (summary, fig_pie, fig_bar, fig_trend, fig_conf)

    except Exception as e:
        return (f"Error creating analytics: {e}", None, None, None, None)

# ---------- Utility UI functions ----------
def list_csv_files():
    files = [f for f in os.listdir(REVIEWS_DIR) if f.endswith('.csv')]
    if not files:
        return "No CSV files in reviews directory."
    out = "CSV files:\n"
    for f in files:
        p = os.path.join(REVIEWS_DIR, f)
        mod = datetime.fromtimestamp(os.path.getmtime(p)).strftime("%Y-%m-%d %H:%M:%S")
        out += f"{f} (modified: {mod})\n"
    return out

def export_reviews_csv():
    # returns absolute path to exported CSV (copy with timestamp)
    if not os.path.exists(REVIEWS_CSV):
        return "No reviews to export."
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    export_path = os.path.join(REVIEWS_DIR, f"customer_reviews_export_{ts}.csv")
    import shutil
    shutil.copy2(REVIEWS_CSV, export_path)
    return f"Exported to: {os.path.abspath(export_path)}"

def load_history_table():
    if not os.path.exists(REVIEWS_CSV):
        return pd.DataFrame()
    df = pd.read_csv(REVIEWS_CSV)
    if df.empty:
        return pd.DataFrame()
    df_display = df[['timestamp', 'image_name', 'dominant_emotion', 'confidence', 'feedback_type']].copy()
    df_display.rename(columns={
        'timestamp': 'Timestamp',
        'image_name': 'Image',
        'dominant_emotion': 'Emotion',
        'confidence': 'Confidence',
        'feedback_type': 'Feedback'
    }, inplace=True)
    return df_display.tail(50)

# ---------- Gradio UI ----------
css = """
body { font-family: Inter, Arial, sans-serif; }
.gradio-container { max-width: 1100px; margin: auto; }
.title { color: #2E8B57; text-align: center; margin-bottom: 10px; }
.card { border-radius: 12px; padding: 12px; background: #fff9f1; box-shadow: 0 4px 10px rgba(0,0,0,0.03);}
"""

with gr.Blocks(title="🍽️ Restaurant Sentiment Dashboard", css=css) as demo:
    gr.Markdown("<h1 class='title'>🍽️ Restaurant Customer Sentiment System</h1>")

    with gr.Tabs():
        # --- Analyze Tab ---
        with gr.Tab("📸 Analyze (Upload / Webcam)"):
            gr.Markdown("Upload a photo or take one with your webcam. Single-frame capture (not continuous live).")
            with gr.Row():
                with gr.Column(scale=1):
                    input_img = gr.Image(label="📷 Upload or Capture (webcam)", type="pil")
                    webcam_img = gr.Image(label="Or use Webcam (capture still)", type="pil")
                    analyze_btn = gr.Button("🔍 Analyze Expression", variant="primary")
                    # option to save image locally (if user wants)
                with gr.Column(scale=1):
                    emotion_output = gr.Textbox(label="🎭 Emotion Analysis", lines=3)
                    command_output = gr.Textbox(label="🧾 Restaurant Command", lines=2)
                    recommendation_output = gr.Textbox(label="💡 Staff Recommendation", lines=2)
                    dominant_hidden = gr.Textbox(visible=False)  # store dominant emotion string

            # Use both inputs: prioritize webcam capture if available; otherwise upload.
            def analyze_wrapper(upload_img, webcam_img):
                img = webcam_img if webcam_img is not None else upload_img
                emotion_details, command_text, recommendation_text, dominant = detect_expression(img)
                # populate hidden textbox with dominant for further use
                return emotion_details, command_text, recommendation_text, dominant

            analyze_btn.click(fn=analyze_wrapper, inputs=[input_img, webcam_img],
                              outputs=[emotion_output, command_output, recommendation_output, dominant_hidden])

        # --- Dashboard Tab ---
        with gr.Tab("📊 Analytics Dashboard"):
            with gr.Row():
                refresh_btn = gr.Button("🔄 Refresh Analytics", variant="primary")
                export_btn = gr.Button("📥 Export CSV copy", variant="secondary")
                list_btn = gr.Button("📁 List CSV Files", variant="secondary")
            summary_md = gr.Markdown("Click refresh to load analytics.")
            pie_plot = gr.Plot()
            bar_plot = gr.Plot()
            trend_plot = gr.Plot()
            conf_plot = gr.Plot()
            csv_list_box = gr.Textbox(label="CSV Files", lines=5)

            def refresh_analytics():
                summary, f1, f2, f3, f4 = create_analytics()
                return summary, f1, f2, f3 or go.Figure(), f4

            refresh_btn.click(fn=refresh_analytics, outputs=[summary_md, pie_plot, bar_plot, trend_plot, conf_plot])
            export_btn.click(fn=export_reviews_csv, outputs=csv_list_box)
            list_btn.click(fn=list_csv_files, outputs=csv_list_box)

        # --- History Tab ---
        with gr.Tab("📋 Review History"):
            gr.Markdown("Latest stored reviews (CSV).")
            history_btn = gr.Button("🗂️ Load Review History", variant="primary")
            history_table = gr.Dataframe(headers=["Timestamp", "Image", "Emotion", "Confidence", "Feedback"], interactive=False)

            history_btn.click(fn=load_history_table, outputs=history_table)

        # --- Upload CSV & Analyze Tab ---
        with gr.Tab("📂 Upload CSV & Analyze"):
            gr.Markdown("Upload a historical CSV exported from other systems (must contain: dominant_emotion, confidence, feedback_type, timestamp columns).")
            csv_file = gr.File(file_types=[".csv"], label="Upload CSV")
            load_csv_btn = gr.Button("📊 Load & Analyze CSV", variant="primary")
            csv_summary = gr.Markdown("Upload CSV to view summary.")
            csv_pie = gr.Plot()
            csv_bar = gr.Plot()
            csv_conf = gr.Plot()
            csv_timeline = gr.Plot()

            def analyze_uploaded_csv(uploaded):
                if uploaded is None:
                    return "No file uploaded", None, None, None, None
                try:
                    df = pd.read_csv(uploaded.name)
                    # quick validation
                    required = {'dominant_emotion', 'confidence', 'feedback_type', 'timestamp'}
                    if not required.issubset(set(df.columns)):
                        return f"CSV missing required columns. Required: {required}", None, None, None, None
                    # summary
                    total = len(df)
                    pos = len(df[df['feedback_type']=="Positive"])
                    neg = len(df[df['feedback_type']=="Negative"])
                    neu = len(df[df['feedback_type']=="Neutral"])
                    avg_conf = df['confidence'].mean()
                    summary = f"Uploaded: {os.path.basename(uploaded.name)}\nTotal: {total}\nPositive:{pos} Negative:{neg} Neutral:{neu}\nAvg Conf: {avg_conf:.1f}%"

                    pie = px.pie(names=['Positive','Negative','Neutral'], values=[pos,neg,neu], title="Sentiment Distribution (Uploaded)")
                    emo = df['dominant_emotion'].value_counts().reset_index()
                    emo.columns = ['Emotion','Count']
                    bar = px.bar(emo, x='Emotion', y='Count', title="Emotion Breakdown (Uploaded)")
                    conf = px.histogram(df, x='confidence', nbins=20, title="Confidence Distribution (Uploaded)")
                    # timeline (if timestamp available)
                    try:
                        df['ts'] = pd.to_datetime(df['timestamp'])
                        daily = df.groupby(df['ts'].dt.date).size()
                        timeline = px.line(x=daily.index, y=daily.values, title="Reviews per Day")
                    except Exception:
                        timeline = go.Figure()
                    return summary, pie, bar, conf, timeline
                except Exception as e:
                    return f"Failed loading CSV: {e}", None, None, None, None

            load_csv_btn.click(fn=analyze_uploaded_csv, inputs=csv_file, outputs=[csv_summary, csv_pie, csv_bar, csv_conf, csv_timeline])

    gr.Markdown("Made with ❤️ for restaurants — Detects single-frame facial expressions, stores data for analytics, and provides staff recommendations.")

# Launch app
if __name__ == "__main__":
    demo.launch(share=True, debug=True)

26-01-20 14:05:47 - Directory /root/.deepface has been created
26-01-20 14:05:47 - Directory /root/.deepface/weights has been created
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d40edcbc6ee61e57fb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


26-01-20 14:08:13 - 🔗 facial_expression_model_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5 to /root/.deepface/weights/facial_expression_model_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 33.5MB/s]
